In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import math

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import subprocess
import cv2
from scipy import ndimage  # for image rotation
import statistics

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

In [ ]:
import sys
print(sys.modules['keras_retinanet'].__file__)
print(sys.modules['keras_resnet'].__file__)
print(sys.modules['keras'].__file__)

In [ ]:

circuitBoard_model = keras.models.load_model('../snapshots/circuit_annotation180322CircuitBoardRF/circuit_resnet50_csv_30.h5', custom_objects=custom_objects)
rfHead_model = keras.models.load_model('../snapshots/circuit_rf2/circuit_resnet50_csv_40.h5', custom_objects=custom_objects)


## configure the features for the circuit

In [ ]:
# using hole1 as the origin

x0 = 1104
y0 = 478
ORIGIN_LABEL_ID = 0

features = [["hole1", 1104-x0, 478-y0, 25],
           ["hole2", 388-x0, 731-y0, 50],
           ["rfSocket", 1150-x0, 328-y0, 30],
           ["tab1", 1037-x0, 284-y0, 25],
           ["tab2", 578-x0, 285-y0, 25]
           ]

numOrigins = 0
numFeatures = numOrigins + len(features)

print(numFeatures)

In [ ]:
import numpy as np
import math

initAngle = np.zeros(numFeatures, dtype=float)
index = numOrigins
for feature in features:
    initAngle[index] = math.atan2(feature[2],feature[1]) * 180.0/math.pi
    index = index+1

print(initAngle)

In [ ]:
def get2DPose(inFileName):
    image = cv2.imread(inFileName)
    print(image.shape)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = validation_generator.preprocess_image(image)
    print(image.shape)
    image, scale = validation_generator.resize_image(image)
    print(image.shape)
    print(scale)
    
    # process image
    start = time.time()
    detections = None
    _, _, detections = circuitBoard_model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
    # correct for image scale
    detections[0, :, :4] /= scale
    
    b = None
    foundTarget = False
    angle = None
    centerX = np.zeros(numFeatures, dtype=int)
    centerY = np.zeros(numFeatures, dtype=int)

    # visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < 0.5 or centerX[label] != 0:
            continue
            
        b = detections[0, idx, :4].astype(int)
        if label == ORIGIN_LABEL_ID: # using hole1 as the origin
            foundTarget = True
            originX = (b[0]+b[2])/2
            originY = (b[1]+b[3])/2
           
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 3)
        caption = "{} {:.3f}".format(validation_generator.label_to_name(label), score)
        cv2.putText(draw, caption, (b[0], b[1] - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
        cv2.putText(draw, caption, (b[0], b[1] - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

        centerX[label] = (b[2]+b[0])/2
        centerY[label] = (b[3]+b[1])/2
        print(label, centerX[label], centerY[label], score)
        
    
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

    if not foundTarget:
        return -999, -999, -999
    
    
    print("origin and angle", originX, originY, angle)
    
    if not foundTarget:
        angleAvg = angle
    else:
        angleList = [] 
        for labelID in range(numOrigins+1, numFeatures, 1):
            if centerX[labelID] != 0 and centerY[labelID] != 0:
                featureAngle = (math.atan2(centerY[labelID] - originY, centerX[labelID] -originX)) * 180.0 / math.pi - initAngle[labelID]
                if featureAngle < -180:
                    featureAngle = featureAngle + 360
                elif featureAngle > 180:
                    featureAngle = 360 - featureAngle
                print("featureAngle:", labelID, centerX[labelID], centerY[labelID], featureAngle)
                angleList.append(featureAngle)
    
        angleMedian = statistics.median(angleList)
    print("returning", originX, originY, angleMedian)
    return originX, originY, angleMedian

In [ ]:
def getRFHeadPose(inFileName):
    
    image = cv2.imread(inFileName)
    # take only the center 500 pixel square
    # convert undistorted-$i.png -gravity center -crop 500x500+55-5 rf$i.png
    image = image[223:723, 443:943, :]
    print(image.shape)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = validation_generator.preprocess_image(image)
    print(image.shape)
    image, scale = validation_generator.resize_image(image)
    print(image.shape)
    print(scale)
    
    # process image
    start = time.time()
    detections = None
    _, _, detections = rfHead_model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
    # correct for image scale
    detections[0, :, :4] /= scale
    
    b = None
    returnX = None
    
    # visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < 0.5:
            continue
            
        b = detections[0, idx, :4].astype(int)
        color = (0, 0, 255) if label == 0 else (255,0,0)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), color, 3)
        caption = "{:.2f}".format(score)
        cv2.putText(draw, caption, (b[0], b[1] - 2), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 3)
        cv2.putText(draw, caption, (b[0], b[1] - 2), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)

        centerX = (b[2]+b[0])/2
        centerY = (b[3]+b[1])/2
        print(label, centerX, centerY, score)
        if label == 0 and returnX == None:
            returnX = centerX
            returnY = centerY
        
    
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
    
    if returnX != None:
        print("returning", returnX, returnY)
        return returnX, returnY
    else:
        print("did not find any")
        return -999, -999


# Create Server to respond to identification request

In [ ]:
import socket
import time

HOST = "192.168.1.116" #The remote host
PORT = 3005 # The same port as used by the server
print("Starting 2d id program")

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

validation_generator = CSVGenerator(
                '/media/kai/Data1/robotPrototype/data/MLRotation/annotation180322CircuitBoardRF.txt',
                '/media/kai/Data1/robotPrototype/data/MLRotation/classname180322CircuitBoardRF.txt',
                val_image_data_generator,
                batch_size=10
            )



In [ ]:
finishedLoop = False
inFileName = ''
xPixel = 0
yPixel = 0
a = 0.0
returnVal_circuit = None
returnVal_rfHead = None

while finishedLoop == False:
    print("Waiting for connections...")
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((HOST, PORT)) # Bind to the port 
    s.listen(5) # Now wait for client connection.
    c, addr = s.accept() # Establish connection with client.
    print("Connected")
    foundError = False
    finishedInner = False
    while foundError == False and finishedInner == False:
        try:
            msg = c.recv(40).decode("utf-8").strip()
            if msg != "":
                print("received: " + msg)
            time.sleep(1)
            if msg == "acquireCircuitOrigin":
                print("acquiring circuit origin ...")
                inFileName = "/media/kai-xps/demo/ur3Control/tmp/undistorted.png"
                xPixel,yPixel,a = get2DPose(inFileName)
                if xPixel == -999:
                    returnVal = "None Found"
                else:    
                    x = (-0.121 * xPixel + 69.02)/1000.0
                    y = (-0.1216 * yPixel + 174.0)/1000.0
                    z = -24/1000.0
                    rx = 0.0
                    ry = 0.0
                    rz = a / 180.0 * math.pi
                    returnVal_circuit = "{0:8.4f}, {1:8.4f}, {2:8.4f}, {3:6.3f}, {4:6.3f}, {5:6.3f}".format(x, y, z, rx, ry, rz)
                print("sending: ", returnVal_circuit)
                c.send(returnVal_circuit.encode())
                finishedInner = True
            elif msg == "getCircuitOrigin":
                returnString = "({})\n".format(returnVal_circuit)
                c.send(returnString.encode())
                print(returnString)
                finishedInner = True
            elif msg == "acquireRFHead":
                print("acquiring rf connector head ...")
                inFileName = "/media/kai-xps/demo/ur3Control/tmp/undistorted.png"
                xPixel, yPixel = getRFHeadPose(inFileName)
                if xPixel < 0:
                    returnVal_rfHead = "-999, -999, -999, 0, 0, 0"
                else:
                    # need to convert from pixel to ur3 coordinate
                    returnVal_rfHead = "{0:8.4f}, {1:8.4f}, 0, 0, 0, 0".format(xPixel, yPixel)
                c.send(result.encode())
                print(result)
                finishedInner = True
            elif msg == "getRFHead":
                returnString = "({})\n".format(returnVal_rfHead)
                c.send(returnString.encode())
                print(returnString)
                finishedInner = True
            elif msg == "end":
                finishedLoop = True
                finishedInner = True
        except socket.error as socketError:
            print(socketError)
            foundError = True
    c.close()
    s.close()

In [ ]:
c.close()
s.close()

# Tests

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

validation_generator = CSVGenerator(
                '/media/kai/Data1/robotPrototype/data/MLRotation/annotation180322CircuitBoardRF.txt',
                '/media/kai/Data1/robotPrototype/data/MLRotation/classname180322CircuitBoardRF.txt',
                val_image_data_generator,
                batch_size=10
            )

In [ ]:
testFile = "/media/kai/Data1/robotPrototype/data/180320RFConnectorDispensor/undistorted-14.png"
result = getRFHeadPose(testFile)

In [ ]:
testFile = "/media/kai/Data1/robotPrototype/data/MLRotation/180322CircuitBoardRF/val12.png"
#testFile = "/media/kai/Data1/robotPrototype/data/180109OriginInferenceIssue/undistorted.png"
#testFile = "/media/kai-laptop/study/robot_prototype/ur3Control/tmp/undistorted.png"
#testFile = "/media/kai-xps/demo/ur3Control/tmp/undistorted.png"
result = get2DPose(testFile)
# result = getRFHeadPose(testFile)

In [ ]:
xPixel = 880
yPixel = 235
x = (-0.121 * xPixel + 69.332)/1000.0
y = (-0.1216 * yPixel + 174.75)/1000.0

print(x,y)

In [ ]:
# rotate image and display
from scipy import ndimage

#img = cv2.imread('/media/kai/Data1/robotPrototype/data/180226ConnectorD/undistorted.png')
img = cv2.imread('/media/kai-xps/demo/ur3Control/tmp/undistorted.png')

#rotation angle in degree
rotated = ndimage.rotate(img, 180)
print(rotated.shape)

xPixel,yPixel,a = get2DPose(rotated)

print(xPixel, yPixel, a)
centerX = rotated.shape[1]/2
centerY = rotated.shape[0]/2

xPixelNew = 2*centerX-xPixel
yPixelNew = 2*centerY-yPixel
aNew = a

print(xPixelNew, yPixelNew, aNew)






In [ ]:
from scipy import ndimage
import cv2
import matplotlib.pyplot as plt

draw = cv2.imread('/media/kai/Data1/robotPrototype/data/180226ConnectorD/undistorted.png')
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

#rotation angle in degree
rotated = ndimage.rotate(draw, 180)

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(rotated)
plt.show()